<a href="https://colab.research.google.com/github/GuidoGiacomoMussini/Text_Mining-Lyrics_Analysis/blob/main/5a_MLM_Model_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm as progress_bar
from collections import Counter
from google.colab import drive
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForMaskedLM
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/Colab\ Notebooks/Text\ Mining/Utils/MLM_Utils.py /content
import MLM_Utils as MLM

In [3]:
#bert for italian language
bert = 'dbmdz/bert-base-italian-cased'
bert = "dbmdz/bert-base-italian-xxl-cased"

tokenizer = BertTokenizer.from_pretrained(bert)
model = BertForMaskedLM.from_pretrained(bert)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Data

In [4]:
df =  pd.read_hdf(r'/content/drive/MyDrive/Colab Notebooks/Text Mining/Files/lyrics_topic_detection.h5', key='data')

df.head(2)

,Author,Title,Lyrics,verses,processed_lyrics,processed_verses,POS,important_POS,common_words,topic_similarity,topic
0,Fabrizio De André,La Guerra Di Piero,dormi sepolto in un campo di grano non è la ro...,"[dormi sepolto in un campo di grano, non è la...",dormire seppellire campo Grano rosa Tulipano f...,"[dormire seppellire campo Grano, rosa Tulipan...","[[VERB, VERB, ADP, DET, NOUN, ADP, NOUN], [ADV...","[campo, fan, veglia, ombra, sponda, luccio, ca...","[tempo, terra, campo, fan, veglia, ombra, inve...","{'guerra': 0.178, 'morte': 0.173, 'dio': 0.168...",guerra
1,Fabrizio De André,Don Raffaè,io mi chiamo pasquale cafiero e son brigadiero...,"[io mi chiamo pasquale cafiero, e son brigadi...",chere pasquale Cafiero son brigadiero carcere ...,"[chere pasquale Cafiero, son brigadiero carce...","[[PRON, PRON, VERB, ADJ, ADJ], [CCONJ, AUX, NO...","[brigadiero, carcere, cafiero, catenaccio, ser...","[cafè, carcere, ricetta, cumpagno, fortuna, uo...","{'dio': 0.182, 'politica': 0.176, 'morte': 0.1...",dio


In [5]:
#filter for De André and remove by hand all the dialets\foreign language songs, which are too influent in the training
removed_list = ['Dolcenera', 'Don Raffaé', 'Prinçesa', 'Crêuza de mä', 'Ottocento', 'Sinan Capudan Pascia', 'S’ì Fosse Foco', '’A Cimma', 'A Dumenega', 'Jamin-a', 'Monti Di Mola',
                'Zirichiltaggia', 'Mègu Megùn', 'Â cúmba', 'A Pittima']
df_train = df[df.Author == 'Fabrizio De André']
df_train = df_train[~df_train['Title'].isin(removed_list)]

In [6]:
#create a list of all the verses
verses_train = [verse.strip() for song in df_train.verses for verse in song]
#list of songs
songs_train = df_train.Lyrics


##bound the verse len btwn 3 an 22 words (empiric)
verses_train = [verse for verse in verses_train if len(verse.split()) > 3 and len(verse.split()) < 23]

In [7]:
#create the dataset. Each word of each verse is masked once
x_train, y_train = MLM.create_masked_pairs(verses_train)
x_train_s, y_train_s = MLM.create_masked_pairs(songs_train)

df_train = pd.DataFrame({'x': x_train, 'y':y_train})
df_train_song = pd.DataFrame({'x': x_train_s, 'y':y_train_s})
df_train.sample(5)

,x,y
10817,l'ho chiuso nei miei [MASK],l'ho chiuso nei miei occhi
770,cumpagno di cella precisa [MASK] mammà,cumpagno di cella precisa a mammà
19892,un po' di tempo fa [MASK] era senza compagnia,un po' di tempo fa nancy era senza compagnia
15661,per conquistarmi il cielo per guadagnarmi [MAS...,per conquistarmi il cielo per guadagnarmi il sole
13639,avrà le tue [MASK] e i tuoi occhi,avrà le tue labbra e i tuoi occhi


# Model

In [8]:
#select to train at verse-level or song-level

#dataset_train = MLM.BertDataset(df_train_song, tokenizer, 100)
dataset_train = MLM.BertDataset(df_train, tokenizer, 50)

dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
num_epochs = 20
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
model.train()
for epoch in progress_bar(range(num_epochs)):
    total_loss = 0
    for batch in progress_bar(dataloader_train):

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader_train)
    print(f"Epoch: {epoch}, Loss: {avg_loss}")

In [11]:
model.to("cpu")
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/Text Mining/Models/De_Andre_MLM_Model_100.pth')

In [12]:
!rm /content/MLM_Utils.py